In [0]:
# Configuração de Conexão ao Banco de Dados PostgreSQL
jdbc_url = "jdbc:postgresql://dpg-cuokht5ds78s738lrhs0-a.oregon-postgres.render.com/databricks_2094"
db_properties = {
    "user": "databricks_2094_user",
    "password": "KggDjPvWQZHiOrkWVMFO7ocyZwbs1AD8",
    "driver": "org.postgresql.Driver"
}

# Nome da tabela no PostgreSQL e no Databricks
postgres_table = "transactions"
databricks_table = "bronze.transactions"

# 1. Obter o último 'transaction_date' da tabela gerenciada no Databricks
last_transaction_date = spark.sql(f"""
    SELECT MAX(transaction_date) AS last_transaction_date 
    FROM {databricks_table}
    """).collect()[0]["last_transaction_date"]

# Verificar se há um 'last_transaction_date' válido
if last_transaction_date is None:
    print("Nenhuma dado encontrado na tabela gerenciada. Carregando todos os dados do PostgreSQL.")
    query = f"(SELECT * FROM {postgres_table}) AS t"
else:
    print(f"(Última transação encontrada: {last_transaction_date}.Carregando apenas os dados novos.")
    query = f"""
        (SELECT *
         FROM {postgres_table}
         WHERE transaction_date > '{last_transaction_date}') AS t 
        """        

# Carregar apenas os dados novos do PostgreSQL para um DataFrame Spark
new_transactions_df = (
    spark.read.format("jdbc")
    .option("url", jdbc_url)
    .option("dbtable", postgres_table)
    .option("user", db_properties["user"])
    .option("password", db_properties["password"])
    .option("driver", db_properties["driver"])
    .load()
    )

# Verificar se há dados novos para inserir
if new_transactions_df.count() > 0:
    print(f"Inserindo {new_transactions_df.count()} novos registros na tabela gerenciada.")

# 3. Inserir dados novos na tabela gerenciada
    new_transactions_df.write.format("delta").mode("append").saveAsTable(databricks_table)

    print("Novos dados inseridos com sucesso!")
else:
    print("Nenhum dado novo encontrado para inserção.")



(Última transação encontrada: 2025-01-18 23:53:54.Carregando apenas os dados novos.
Inserindo 2000 novos registros na tabela gerenciada.
Novos dados inseridos com sucesso!
